# Lesson 03
## Парсинг данных. HTML, Beautiful Soap

#### Вариант 1

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    Наименование вакансии.
    Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
    Ссылку на саму вакансию.
    Сайт, откуда собрана вакансия.

По желанию можно добавить ещё параметры вакансии (например, работодателя и расположение). Структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas. Сохраните в json либо csv.

In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep
import pandas as pd

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true'

headers = {
    'User-Agent': USER_AGENT,
}

In [3]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [4]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true
Cтраница №2, ссылка: https://hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&page=3&hhtmFrom=vacancy_search_list
Cтраница №5, ссылка: https://hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&page=4&hhtmFrom=vacancy_search_list
Cтраница №6, ссылка: https://hh

In [5]:
len(result)

118

In [6]:
result[:1]

[{'vacancy_name': 'Java-разработчик (удаленно/Сербия)',
  'vacancy_salary': 'до 450\u202f000 руб.',
  'min_salary': None,
  'max_salary': 450000,
  'currency_salary': 'руб.',
  'vacancy_link': 'https://hh.ru/vacancy/77463926?from=vacancy_search_list&query=Java',
  'vacancy_source': 'hh.ru'}]

In [7]:
dfres=pd.DataFrame(result)
dfres

,vacancy_name,vacancy_salary,min_salary,max_salary,currency_salary,vacancy_link,vacancy_source
0,Java-разработчик (удаленно/Сербия),до 450 000 руб.,NaN,450000.0,руб.,https://hh.ru/vacancy/77463926?from=vacancy_se...,hh.ru
1,Разработчик Java/Jakarta (удаленно),None,NaN,NaN,None,https://hh.ru/vacancy/77235716?from=vacancy_se...,hh.ru
2,Senior Java Backend,до 450 000 руб.,NaN,450000.0,руб.,https://hh.ru/vacancy/77617915?from=vacancy_se...,hh.ru
3,Java-разработчик,None,NaN,NaN,None,https://hh.ru/vacancy/77524857?from=vacancy_se...,hh.ru
4,Java Backend Разработчик (Middle),None,NaN,NaN,None,https://hh.ru/vacancy/71215385?from=vacancy_se...,hh.ru
...,...,...,...,...,...,...,...
113,Senior Java-разработчик,None,NaN,NaN,None,https://hh.ru/vacancy/76601234?from=vacancy_se...,hh.ru
114,QA automation engineer,None,NaN,NaN,None,https://hh.ru/vacancy/74370491?from=vacancy_se...,hh.ru
115,QA automation,None,NaN,NaN,None,https://hh.ru/vacancy/74371380?from=vacancy_se...,hh.ru
116,DevOps engineer,None,NaN,NaN,None,https://hh.ru/vacancy/76559496?from=vacancy_se...,hh.ru


In [8]:
dfres.to_csv('hh_java.csv', encoding='utf8')